In [ ]:
!pip install gradio


In [ ]:
!pip install pydantic

In [ ]:
!pip install chromadb

In [ ]:
!pip install pypdf

In [ ]:
!pip install -qqq torch --progress-bar off


In [ ]:
!pip install -qqq langchain --progress-bar off



In [ ]:
!pip install transformers


In [ ]:
!pip install -qqq optimum --progress-bar off


In [ ]:
!pip install -qqq auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off


In [ ]:
!pip install accelerate --q

In [ ]:
!pip install -U sentence_transformers --q

In [ ]:
!pip install --upgrade transformers optimum gradio


In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import HuggingFaceHub
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import optimum
import auto_gptq

In [ ]:
import gradio as gr

In [ ]:
def summarize(cv_path, industry):
    loader = PyPDFLoader(cv_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
    texts = text_splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True}
    )

    db = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory="db")
    MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="cuda"
    )

    generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
    generation_config.max_new_tokens = 1024
    generation_config.temperature = 0.0001
    generation_config.top_p = 0.95
    generation_config.do_sample = True
    generation_config.repetition_penalty = 1.15

    text_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        generation_config=generation_config,
    )

    llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

    template = f"""<s>[INST] <<SYS>> As a Human Resources expert specializing in the {industry} industry,
you are tasked with evaluating the following candidate's CV.
Your goal is to assess their suitability for a role that requires innovation, teamwork, and leadership skills.
  <</SYS>> {{context}} {{question}} [/INST]"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt},
    )

    result = qa_chain(
        "Based on the CV's contents, write a comprehensive summary that highlights the candidate's key qualifications, experiences, and skills. Additionally, formulate a set of five interview questions designed to explore the candidate's strengths and areas for growth in relation to the job's requirements."
    )
    return result["result"].strip()



In [ ]:
def gradio_interface(uploaded_file, industry):
    if uploaded_file is None:
        return "No file uploaded. Please upload a PDF file."
    summary = summarize(uploaded_file.name, industry)
    return summary


industry_choices = [
    "Technology", "Healthcare", "Financial Services", "Retail", "Consumer Goods",
    "Energy", "Manufacturing", "Telecommunications", "Agriculture", "Construction",
    "Education", "Transportation and Logistics", "Pharmaceuticals", "Real Estate",
    "Hospitality and Tourism", "Media and Entertainment", "Automotive",
    "Aerospace and Defense", "Food and Beverage", "Information Services"
]

iface = gr.Interface(
    fn=gradio_interface,
    inputs=[
        gr.File(label="Upload PDF"),
        gr.Dropdown(choices=industry_choices, label="Industry")
    ],
    outputs=gr.Textbox(label="Summary and Questions"),
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://25e2892feae3a1a672.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
